In [1]:
import numpy as np
import pandas as pd

In [45]:
# read table

dataPayment = '../Datasets/Employee_Payroll.csv'
payroll = pd.read_csv(dataPayment)

# set default value to 0 for NaN numerical data
numeric_cols = payroll.select_dtypes(include=[np.number]).columns
payroll[numeric_cols] = payroll[numeric_cols].clip(lower=0)


payroll.fillna(0, inplace=True)
payroll['Office'] = payroll['Office'].astype(int)

# define column for 1/4 year discretization
payroll['Fiscal Quarter'] = payroll['Fiscal Quarter']*0.25 - 0.25
payroll['Fiscal Period'] = payroll['Fiscal Year'] + payroll['Fiscal Quarter']

pd.to_datetime(payroll['Original Hire Date'])

# parse hire date to get hire year
payroll['Original Hire Date'] = payroll['Original Hire Date'].str.split('/').str[2]
payroll['Original Hire Date'] = payroll['Original Hire Date'].astype(int)

_work_year = payroll["Fiscal Period"] - payroll['Original Hire Date']

KeyError: 'Office'

In [ ]:
def typeconverter(col):
    if col == "Iris-setosa":
        return 0
    if col == "Iris-versicolor":
        return 1
    if col == "Iris-virginica":
        return 2

col_names = ['sepal length', 'sepal width', 'petal_length', 'petal_width', 'type']
data = pd.read_csv('../Datasets/iris.data', skiprows=1, header=None, names=col_names, converters={
    'type': typeconverter
})
data.head(4)

In [23]:
# specify table column included

data = payroll[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Position ID', 'Base Pay']]

# add Working Year Column
payroll["Working Year"] = _work_year
data = payroll[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Position ID', 'Base Pay', 'Working Year']]
data.head(4)

,Fiscal Year,Fiscal Period,Job Code,Position ID,Base Pay,Working Year
0,2016,2016.00,1172,9510200,20088.00,11.00
1,2016,2016.25,1172,9510200,23436.00,11.25
2,2016,2016.50,1172,9510200,20422.82,11.50
3,2016,2016.75,1172,9510200,23904.80,11.75


### Node Class

In [3]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        self.value = value # leaf node

### Tree Class

In [4]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

### Train-Test-Split

In [42]:
X = data.iloc[:,3:-1].values
Y = data.iloc[:,-2].values.reshape(-1, 1)

# X = data[['Fiscal Year', 'Fiscal Period', 'Job Code', 'Position ID', 'Working Year']]
# Y = data[['Base Pay']]

X = data.iloc[:].values
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

X

array([[2.016000e+03, 2.016000e+03, 1.172000e+03, 9.510200e+06,
        2.008800e+04, 1.100000e+01],
       [2.016000e+03, 2.016250e+03, 1.172000e+03, 9.510200e+06,
        2.343600e+04, 1.125000e+01],
       [2.016000e+03, 2.016500e+03, 1.172000e+03, 9.510200e+06,
        2.042282e+04, 1.150000e+01],
       ...,
       [2.018000e+03, 2.018000e+03, 1.567000e+03, 9.512421e+06,
        1.921740e+04, 3.300000e+01],
       [2.018000e+03, 2.018000e+03, 5.296000e+03, 9.519643e+06,
        9.698800e+03, 2.000000e+00],
       [2.018000e+03, 2.018250e+03, 4.800000e+01, 1.400088e+06,
        1.287244e+04, 7.250000e+00]])

### Fit the model

In [43]:
model = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
model.fit(X_train, Y_train)
model.print_tree()

KeyboardInterrupt: 

### Accuracy Test

In [7]:
Y_pred = model.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.8666666666666667